In [34]:
import pandas as pd
import plotly.graph_objects as go

# Ler o arquivo CSV
df = pd.read_csv('dados.csv')

# Histograma para ilustrar a distribuição de clientes por idade

In [35]:
# Contar a frequência para cada idade
age_counts = df['Idade'].value_counts().sort_index()

# Criar a figura
fig = go.Figure()

# Adicionar barras
fig.add_trace(go.Bar(
    x=age_counts.index,
    y=age_counts.values,
    marker_color='skyblue',
    marker_line_color='black',
    marker_line_width=1,
    hovertemplate='Idade: %{x} anos<br>Número de Clientes: %{y}<extra></extra>'
))

# Personalizar o layout
fig.update_layout(
    title={
        'text': 'Distribuição de Clientes por Idade',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20}
    },
    xaxis_title={
        'text': 'Idade (anos)',
        'font': {'size': 14}
    },
    yaxis_title={
        'text': 'Número de Clientes',
        'font': {'size': 14}
    },
    xaxis={
        'dtick': 1,  # Forçar marca de tick para cada idade
        'tick0': age_counts.index.min(),  # Iniciar da idade mínima
    },
    template='plotly_white',
    showlegend=False,
    bargap=0.2
)

# Exibir o gráfico
fig.show()

# Percentual de clientes inadimplentes

In [36]:
import plotly.express as px

# Calcular estatísticas
total_customers = len(df)
null_count = df['Inadimplente'].isna().sum()
delinquent_count = df['Inadimplente'].eq(1).sum()
non_delinquent_count = df['Inadimplente'].eq(0).sum()

# Criar DataFrame de análise
analysis_df = pd.DataFrame({
    'Status': ['Adimplente', 'Inadimplente', 'Sem Informação'],
    'Quantidade': [non_delinquent_count, delinquent_count, null_count],
    'Percentual': [
        (non_delinquent_count/total_customers),
        (delinquent_count/total_customers),
        (null_count/total_customers)
    ]
})

# Calcular taxas para diferentes abordagens
known_status_df = df[df['Inadimplente'].notna()]
known_total = len(known_status_df)
rates_df = pd.DataFrame({
    'Abordagem': [
        'Conservadora (excluindo NULLs)',
        'Melhor Cenário (NULLs como adimplentes)',
        'Pior Cenário (NULLs como inadimplentes)'
    ],
    'Taxa de Inadimplência': [
        delinquent_count / known_total,
        delinquent_count / total_customers,
        (delinquent_count + null_count) / total_customers
    ]
})

def style_dataframe(df):
    return df.style\
        .format({
            'Quantidade': '{:,.0f}',
            'Percentual': '{:.2%}',
            'Taxa de Inadimplência': '{:.2%}'
        })\
        .set_properties(**{
            'background-color': '#f8f9fa',
            'color': 'black',
            'border-color': '#dee2e6',
            'font-family': 'Arial, sans-serif',
            'padding': '12px',
            'text-align': 'center'
        })\
        .set_table_styles([{
            'selector': 'th',
            'props': [
                ('background-color', '#4a90e2'),
                ('color', 'white'),
                ('font-weight', 'bold'),
                ('text-align', 'center'),
                ('padding', '12px')
            ]
        }])\
        .hide(axis='index')

# Exibir total e tabelas
print(f"Total de Clientes: {total_customers:,}")
display(style_dataframe(analysis_df))
display(style_dataframe(rates_df))

# Criar gráfico de rosca
fig = px.pie(
    analysis_df,
    values='Quantidade',
    names='Status',
    title='Distribuição do Status de Inadimplência',
    color_discrete_sequence=['#2ecc71', '#e74c3c', '#95a5a6'],
    hole=0.4
)

fig.update_layout(
    title={
        'text': 'Distribuição do Status de Inadimplência',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 16}
    }
)

fig.update_traces(
    textinfo='percent+value',
    hovertemplate='Status: %{label}<br>Quantidade: %{value:,.0f}<br>Percentual: %{percent}<extra></extra>'
)

fig.show()

Total de Clientes: 850


Status,Quantidade,Percentual
Adimplente,517,60.82%
Inadimplente,183,21.53%
Sem Informação,150,17.65%


Abordagem,Taxa de Inadimplência
Conservadora (excluindo NULLs),26.14%
Melhor Cenário (NULLs como adimplentes),21.53%
Pior Cenário (NULLs como inadimplentes),39.18%


# Excluíndo outliers e clientes com informações ausentes

A fim de melhorar a qualidade da segmentação, serão removidos os clientes com informação de inadimplência ausente e clientes com padrões muito desviados da média, com o método z-score.

In [37]:
from scipy import stats

# Remover as colunas indesejadas
df = df.drop(columns=['Unnamed: 0', 'ID do cliente'])

# Remover linhas com valores ausentes e capturar as linhas removidas para exibição
df_cleaned = df.dropna()
removed_missing_values = df[~df.index.isin(df_cleaned.index)]  # Capturar linhas com valores ausentes

# Remover linhas com outliers usando o método do escore z (limite |3|)
z_scores = stats.zscore(df_cleaned.select_dtypes(include=['float64', 'int64']))
abs_z_scores = abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)

# Filtrar linhas consideradas como outliers e capturar os outliers removidos para exibição
df_no_outliers = df_cleaned[filtered_entries]
removed_outliers = df_cleaned[~filtered_entries]

# Exibir os dados limpos
print("Dados Limpos sem Valores Ausentes e Outliers:")
display(df_no_outliers)

# Exibir os dados que foram excluídos
print("Dados Excluídos devido a Valores Ausentes:")
display(removed_missing_values)

print("Dados Excluídos devido a Outliers:")
display(removed_outliers)

Dados Limpos sem Valores Ausentes e Outliers:


,Idade,Educação,Anos de emprego,Renda,Dívida do cartão,Outras dívidas,Inadimplente,Razão débito-renda
0,41,2,6,19,0.124,1.073,0.0,6.3
1,47,1,26,100,4.582,8.218,0.0,12.8
2,33,2,10,57,6.111,5.802,1.0,20.9
3,29,2,4,19,0.681,0.516,0.0,6.3
6,38,2,4,56,0.442,0.454,0.0,1.6
...,...,...,...,...,...,...,...,...
843,32,2,8,45,0.982,0.683,0.0,3.7
844,41,1,7,43,0.694,1.198,0.0,4.4
846,28,2,7,34,0.359,2.021,0.0,7.0
848,32,1,12,28,0.116,0.696,0.0,2.9


Dados Excluídos devido a Valores Ausentes:


,Idade,Educação,Anos de emprego,Renda,Dívida do cartão,Outras dívidas,Inadimplente,Razão débito-renda
5,40,1,23,81,0.998,7.831,NaN,10.9
8,26,1,5,18,0.575,2.215,NaN,15.5
11,34,2,9,40,0.374,0.266,NaN,1.6
13,46,1,6,30,1.415,3.865,NaN,17.6
15,24,1,1,16,0.185,1.287,NaN,9.2
...,...,...,...,...,...,...,...,...
818,35,2,0,35,2.383,1.957,NaN,12.4
820,37,1,4,24,0.419,2.989,NaN,14.2
825,32,2,12,116,4.027,2.585,NaN,5.7
835,21,3,0,41,2.367,5.628,NaN,19.5


Dados Excluídos devido a Outliers:


,Idade,Educação,Anos de emprego,Renda,Dívida do cartão,Outras dívidas,Inadimplente,Razão débito-renda
4,47,1,31,253,9.308,8.908,0.0,7.2
24,37,4,10,123,3.022,18.257,0.0,17.3
43,37,5,9,177,0.888,9.555,0.0,5.9
51,36,1,11,33,1.266,9.459,0.0,32.5
78,41,2,21,145,3.237,14.453,0.0,12.2
81,30,3,0,65,3.900,15.405,1.0,29.7
82,40,3,18,157,3.326,7.036,0.0,6.6
183,47,1,29,129,20.561,12.076,1.0,25.3
198,47,3,16,221,15.792,23.104,1.0,17.6
207,43,1,25,242,1.636,4.656,0.0,2.6


# Normalização com standard scaler (z-score)

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Selecionar colunas numéricas para normalização
numeric_columns = df_no_outliers.select_dtypes(include=['float64', 'int64']).columns

# Inicializar o StandardScaler
scaler = StandardScaler()

# Ajustar e transformar os dados
# Criar uma cópia para evitar o aviso SettingWithCopyWarning
df_normalized = df_no_outliers.copy()
df_normalized[numeric_columns] = scaler.fit_transform(df_normalized[numeric_columns])

# Exibir os dados normalizados
print("Dados Normalizados usando StandardScaler:")
display(df_normalized)

Dados Normalizados usando StandardScaler:


,Idade,Educação,Anos de emprego,Renda,Dívida do cartão,Outras dívidas,Inadimplente,Razão débito-renda
0,0.840132,0.381458,-0.310551,-0.920158,-0.849592,-0.656402,-0.574439,-0.569768
1,1.606453,-0.775165,2.919567,2.542951,2.406980,2.437016,-0.574439,0.482053
2,-0.181629,0.381458,0.335473,0.704510,3.523915,1.391012,1.740830,1.792783
3,-0.692510,0.381458,-0.633562,-0.920158,-0.442703,-0.897554,-0.574439,-0.569768
6,0.456971,0.381458,-0.633562,0.661756,-0.617293,-0.924397,-0.574439,-1.330315
...,...,...,...,...,...,...,...,...
843,-0.309349,0.381458,0.012461,0.191457,-0.222823,-0.825252,-0.574439,-0.990496
844,0.840132,-0.775165,-0.149045,0.105948,-0.433207,-0.602283,-0.574439,-0.877223
846,-0.820230,0.381458,-0.149045,-0.278842,-0.677924,-0.245967,-0.574439,-0.456495
848,-0.309349,-0.775165,0.658485,-0.535368,-0.855436,-0.819623,-0.574439,-1.119951


# Escolha de 'k' pelo método do cotovelo

In [ ]:
import plotly.express as px
from sklearn.cluster import KMeans
import pandas as pd

# Selecionando apenas colunas numéricas para a clusterização
numeric_data = df_normalized.select_dtypes(include=['float64', 'int64'])

# Método do cotovelo para determinar o número ideal de clusters
inertia_values = []
cluster_range = range(1, 11)  # Verificando valores de k de 1 a 10

for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(numeric_data)
    inertia_values.append(kmeans.inertia_)

# Criando um DataFrame para plotagem
elbow_data = pd.DataFrame({
    'Número de Clusters (k)': list(cluster_range),
    'Inércia': inertia_values
})

# Plotando a curva do cotovelo
fig = px.line(elbow_data, x='Número de Clusters (k)', y='Inércia', markers=True, 
              title='Método do Cotovelo para k Ótimo',
              labels={'Inércia': 'Inércia', 'Número de Clusters (k)': 'Número de Clusters'},
              template='plotly_white')

fig.update_traces(line=dict(dash='dash'), marker=dict(size=8, color='blue'))
fig.update_layout(title_x=0.5, xaxis=dict(dtick=1))

fig.show()

# Executando a clusterização com k-means

In [40]:
from sklearn.cluster import KMeans
import pandas as pd

# Selecionando apenas colunas numéricas para a clusterização
numeric_data = df_normalized.select_dtypes(include=['float64', 'int64'])

# Aplicando k-means com 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42)
df_normalized['Cluster'] = kmeans.fit_predict(numeric_data)

# Exibir o DataFrame resultante com as atribuições de cluster
print("Dados com Atribuições de Cluster:")
display(df_normalized)

# Exibir os centros dos clusters (valores normalizados)
print("Centros dos Clusters (Normalizados):")
normalized_centroids = pd.DataFrame(kmeans.cluster_centers_, columns=numeric_data.columns)
display(normalized_centroids)

# Desnormalizando os centróides para a escala original
denormalized_centroids = scaler.inverse_transform(kmeans.cluster_centers_)

# Arredondando os valores desnormalizados para duas casas decimais
denormalized_centroids_df = pd.DataFrame(denormalized_centroids, columns=numeric_data.columns).round(2)

# Exibir os centros dos clusters (valores originais, arredondados)
print("Centros dos Clusters (Desnormalizados, Arredondados para 2 Casas Decimais):")
display(denormalized_centroids_df)

Dados com Atribuições de Cluster:


,Idade,Educação,Anos de emprego,Renda,Dívida do cartão,Outras dívidas,Inadimplente,Razão débito-renda,Cluster
0,0.840132,0.381458,-0.310551,-0.920158,-0.849592,-0.656402,-0.574439,-0.569768,1
1,1.606453,-0.775165,2.919567,2.542951,2.406980,2.437016,-0.574439,0.482053,2
2,-0.181629,0.381458,0.335473,0.704510,3.523915,1.391012,1.740830,1.792783,2
3,-0.692510,0.381458,-0.633562,-0.920158,-0.442703,-0.897554,-0.574439,-0.569768,1
6,0.456971,0.381458,-0.633562,0.661756,-0.617293,-0.924397,-0.574439,-1.330315,1
...,...,...,...,...,...,...,...,...,...
843,-0.309349,0.381458,0.012461,0.191457,-0.222823,-0.825252,-0.574439,-0.990496,1
844,0.840132,-0.775165,-0.149045,0.105948,-0.433207,-0.602283,-0.574439,-0.877223,1
846,-0.820230,0.381458,-0.149045,-0.278842,-0.677924,-0.245967,-0.574439,-0.456495,1
848,-0.309349,-0.775165,0.658485,-0.535368,-0.855436,-0.819623,-0.574439,-1.119951,1


Centros dos Clusters (Normalizados):


,Idade,Educação,Anos de emprego,Renda,Dívida do cartão,Outras dívidas,Inadimplente,Razão débito-renda
0,-0.476878,0.171163,-0.759411,-0.549250,0.028949,-0.093926,1.545385,0.616551
1,-0.099792,-0.077470,-0.105313,-0.254083,-0.456352,-0.451118,-0.574439,-0.469380
2,0.825835,0.018934,1.165902,1.338488,1.237023,1.363668,-0.177042,0.597983


Centros dos Clusters (Desnormalizados, Arredondados para 2 Casas Decimais):


,Idade,Educação,Anos de emprego,Renda,Dívida do cartão,Outras dívidas,Inadimplente,Razão débito-renda
0,30.69,1.82,3.22,27.68,1.33,2.37,0.92,13.63
1,33.64,1.60,7.27,34.58,0.66,1.55,0.00,6.92
2,40.89,1.69,15.14,71.83,2.98,5.74,0.17,13.52


-O cluster 1 apresenta os clientes mais jovens, com grau mais alto de educação e menos anos de carreira. Sua renda também é a menor e possui maior probabilidade de inadimplência.

-O cluster 2 é um cluster intermediário em idade, anos de emprego e renda, possuíndo grau zero de inadimplência e baixo endividamento.

-O cluster 3 é o mais velho e com maior renda, possuíndo o maior endividamento e tendo grau baixo de inadimplência.